In [1]:
import os
import pandas as pd
import fetchData
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset_purposes = ['train', 'test']

FUSER_airports = ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']

file_types = ['first_position', 'ETD', 'runways', 'LAMP', 'TBFM', 'TFM_track', 'configs',  'MFS']

years = ['2022', '2023']
monthes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31']

In [3]:
data_type = 'FUSER'
NUM_INTERVALS = 12
INTERVAL_LENGTH = 15


In [4]:
def assign_intervals(df, time_col, ref_time_str, interval_length, num_intervals):
    df = df.copy()
    ref_time = pd.to_datetime(ref_time_str, utc=True, errors='coerce')
    df[time_col] = pd.to_datetime(df[time_col], utc=True, errors='coerce')
    df['time_diff'] = (df[time_col] - ref_time).dt.total_seconds() / 60.0
    df = df[(df['time_diff'] >= 0) & (df['time_diff'] < interval_length * num_intervals)]
    df['interval_idx'] = (df['time_diff'] // interval_length).astype(int) + 1
    return df

In [5]:
# lightning_prob and cloud mapping dictionary
lightning_map = {
    'N': 0,
    'L': 1,
    'M': 2,
    'H': 3
}

# cloud category mapping example, you can add more mappings based on real data
cloud_map = {
    'CL': 0,
    'FW': 1,
    'SC': 2,
    'BK': 3,
    'OV': 4
}


In [12]:
def assign_intervals(df, time_col, ref_time, interval_length, num_intervals):
    """Assign flights to time intervals based on ref_time.
       Each interval is of length interval_length minutes, up to num_intervals intervals."""
    df = df.copy()
    ref_time = pd.to_datetime(ref_time, utc=True, errors='coerce')
    df[time_col] = pd.to_datetime(df[time_col], utc=True, errors='coerce')
    df['time_diff'] = (df[time_col] - ref_time).dt.total_seconds() / 60.0
    df = df[(df['time_diff'] >= 0) & (df['time_diff'] < interval_length * num_intervals)]
    df['interval_idx'] = (df['time_diff'] // interval_length).astype(int) + 1
    return df

for dataset_purpose in dataset_purposes:
    for year in years:
        for month in monthes:
            for day in days:
                data_range = f"{year}-{month}-{day}"

                df_dic = {}
                for file_type in file_types:
                    data_list = []
                    for airport in FUSER_airports:
                        file_name = fetchData.get_fuser_file_name(
                            airport=airport,
                            data_range=data_range,
                            file_type=file_type
                        )
                        output_file_name = f"{data_range}"

                        # If the output file already exists and the input also exists, skip
                        if (fetchData.check_output_file_exists(
                                data_type=data_type,
                                dataset_purpose=dataset_purpose,
                                file_name=output_file_name,
                                base_dir='./data')
                            and fetchData.check_input_file_exists(
                                data_type=data_type,
                                dataset_purpose=dataset_purpose,
                                path_level=airport,
                                file_name=file_name,
                                base_dir='F:\cs523\data')):
                            file_path = f"./data/{data_type}/{dataset_purpose}/{data_range}.csv"
                            print(file_path, "already saved, skip!")
                            continue

                        # If output does not exist but input exists, load the data
                        if (not fetchData.check_output_file_exists(
                                data_type=data_type,
                                dataset_purpose=dataset_purpose,
                                file_name=output_file_name,
                                base_dir='./data')
                            and fetchData.check_input_file_exists(
                                data_type=data_type,
                                dataset_purpose=dataset_purpose,
                                path_level=airport,
                                file_name=file_name,
                                base_dir='F:\cs523\data')):

                            data = fetchData.load_data(
                                data_type=data_type,
                                dataset_purpose=dataset_purpose,
                                path_level=airport,
                                file_name=file_name,
                                base_dir='F:\cs523\data'
                            )

                            # Ensure there is an airport_id column
                            if 'airport_id' not in data.columns:
                                data['airport_id'] = airport

                            data_list.append(data)

                    if data_list:
                        df = pd.concat(data_list, ignore_index=True)
                        if 'airport_id' not in df.columns:
                            # If for some reason airport_id is missing, assign it here
                            df['airport_id'] = airport
                        df_dic[file_type] = df
                    else:
                        # If no data found for this file_type, create an empty df with airport_id column
                        df_dic[file_type] = pd.DataFrame(columns=['airport_id'])

                # If configs is empty or no start_time, cannot proceed
                if df_dic['configs'].empty:
                    continue
                if 'start_time' not in df_dic['configs'].columns:
                    continue

                configs_df = df_dic['configs'].dropna(subset=['start_time'])
                configs_df = configs_df.sort_values(['airport_id','start_time'])
                configs_df['start_time'] = pd.to_datetime(configs_df['start_time'], utc=True, errors='coerce')

                # Convert time columns in other datasets to datetime if they exist
                if (not df_dic['runways'].empty) and ('arrival_runway_actual_time' in df_dic['runways'].columns):
                    df_dic['runways']['arrival_runway_actual_time'] = pd.to_datetime(df_dic['runways']['arrival_runway_actual_time'], utc=True, errors='coerce')

                if (not df_dic['ETD'].empty) and ('departure_runway_estimated_time' in df_dic['ETD'].columns):
                    df_dic['ETD']['departure_runway_estimated_time'] = pd.to_datetime(df_dic['ETD']['departure_runway_estimated_time'], utc=True, errors='coerce')

                if (not df_dic['TFM_track'].empty) and ('arrival_runway_estimated_time' in df_dic['TFM_track'].columns):
                    df_dic['TFM_track']['arrival_runway_estimated_time'] = pd.to_datetime(df_dic['TFM_track']['arrival_runway_estimated_time'], utc=True, errors='coerce')

                if (not df_dic['TBFM'].empty) and ('arrival_runway_sta' in df_dic['TBFM'].columns):
                    df_dic['TBFM']['arrival_runway_sta'] = pd.to_datetime(df_dic['TBFM']['arrival_runway_sta'], utc=True, errors='coerce')

                if (not df_dic['LAMP'].empty) and ('timestamp' in df_dic['LAMP'].columns) and ('forecast_timestamp' in df_dic['LAMP'].columns):
                    df_dic['LAMP']['timestamp'] = pd.to_datetime(df_dic['LAMP']['timestamp'], utc=True, errors='coerce')
                    df_dic['LAMP']['forecast_timestamp'] = pd.to_datetime(df_dic['LAMP']['forecast_timestamp'], utc=True, errors='coerce')

                all_records = []

                for (airport_id), cfg_grp in configs_df.groupby('airport_id'):
                    for _, cfg_row in cfg_grp.iterrows():
                        ref_time = cfg_row['start_time']
                        ref_end = ref_time + pd.Timedelta(minutes=INTERVAL_LENGTH*NUM_INTERVALS)

                        # Process arrivals (runways)
                        if ('runways' in df_dic) and (not df_dic['runways'].empty) and ('arrival_runway_actual_time' in df_dic['runways'].columns):
                            arrivals_sub = df_dic['runways'][
                                (df_dic['runways']['airport_id'] == airport_id) &
                                (df_dic['runways']['arrival_runway_actual_time'].notnull())
                            ]
                            arrivals_sub = assign_intervals(arrivals_sub, 'arrival_runway_actual_time', ref_time, INTERVAL_LENGTH, NUM_INTERVALS)
                            arrival_counts = arrivals_sub.groupby('interval_idx')['gufi'].count().reindex(range(1, NUM_INTERVALS+1), fill_value=0)
                        else:
                            arrival_counts = pd.Series(0, index=range(1, NUM_INTERVALS+1))

                        # Process ETD data
                        if ('ETD' in df_dic) and (not df_dic['ETD'].empty) and ('departure_runway_estimated_time' in df_dic['ETD'].columns):
                            ETD_sub = df_dic['ETD'][
                                (df_dic['ETD']['airport_id'] == airport_id) &
                                (df_dic['ETD']['departure_runway_estimated_time'].notnull())
                            ]
                            ETD_sub = assign_intervals(ETD_sub, 'departure_runway_estimated_time', ref_time, INTERVAL_LENGTH, NUM_INTERVALS)
                            ETD_counts = ETD_sub.groupby('interval_idx')['gufi'].count().reindex(range(1, NUM_INTERVALS+1), fill_value=0)
                        else:
                            ETD_counts = pd.Series(0, index=range(1, NUM_INTERVALS+1))

                        # Process TFM data
                        if ('TFM_track' in df_dic) and (not df_dic['TFM_track'].empty) and ('arrival_runway_estimated_time' in df_dic['TFM_track'].columns):
                            TFM_sub = df_dic['TFM_track'][
                                (df_dic['TFM_track']['airport_id'] == airport_id) &
                                (df_dic['TFM_track']['arrival_runway_estimated_time'].notnull())
                            ]
                            TFM_sub = assign_intervals(TFM_sub, 'arrival_runway_estimated_time', ref_time, INTERVAL_LENGTH, NUM_INTERVALS)
                            TFM_counts = TFM_sub.groupby('interval_idx')['gufi'].count().reindex(range(1, NUM_INTERVALS+1), fill_value=0)
                        else:
                            TFM_counts = pd.Series(0, index=range(1, NUM_INTERVALS+1))

                        # Process TBFM data
                        if ('TBFM' in df_dic) and (not df_dic['TBFM'].empty) and ('arrival_runway_sta' in df_dic['TBFM'].columns):
                            TBFM_sub = df_dic['TBFM'][
                                (df_dic['TBFM']['airport_id'] == airport_id) &
                                (df_dic['TBFM']['arrival_runway_sta'].notnull())
                            ]
                            TBFM_sub = assign_intervals(TBFM_sub, 'arrival_runway_sta', ref_time, INTERVAL_LENGTH, NUM_INTERVALS)
                            TBFM_counts = TBFM_sub.groupby('interval_idx')['gufi'].count().reindex(range(1, NUM_INTERVALS+1), fill_value=0)
                        else:
                            TBFM_counts = pd.Series(0, index=range(1, NUM_INTERVALS+1))

                        # Create initial result DataFrame for this ref_time
                        result_df = pd.DataFrame({
                            'airport_id': [airport_id],
                            'ref_time': [ref_time],
                            **{f'interval_{i}_Arrival': [arrival_counts[i]] for i in range(1, NUM_INTERVALS+1)},
                            **{f'interval_{i}_ETD': [ETD_counts[i]] for i in range(1, NUM_INTERVALS+1)},
                            **{f'interval_{i}_TFM': [TFM_counts[i]] for i in range(1, NUM_INTERVALS+1)},
                            **{f'interval_{i}_TBFM': [TBFM_counts[i]] for i in range(1, NUM_INTERVALS+1)},
                        })

                        # Ensure LAMP has airport_id column
                        if 'LAMP' in df_dic and df_dic['LAMP'].empty:
                            df_dic['LAMP'] = pd.DataFrame(columns=['airport_id'])
                        if 'LAMP' in df_dic and 'airport_id' not in df_dic['LAMP'].columns:
                            df_dic['LAMP']['airport_id'] = airport_id

                        # Process LAMP data
                        if 'LAMP' in df_dic and not df_dic['LAMP'].empty and 'airport_id' in df_dic['LAMP'].columns:
                            LAMP_sub = df_dic['LAMP'][(df_dic['LAMP']['airport_id'] == airport_id)].copy()
                        else:
                            LAMP_sub = pd.DataFrame()

                        if not LAMP_sub.empty and 'forecast_timestamp' in LAMP_sub.columns and 'timestamp' in LAMP_sub.columns:
                            # Keep rows where forecast_timestamp - timestamp = 30 minutes
                            LAMP_sub['time_diff'] = LAMP_sub['forecast_timestamp'] - LAMP_sub['timestamp']
                            LAMP_sub = LAMP_sub[LAMP_sub['time_diff'] == pd.Timedelta(minutes=30)]

                            # Filter LAMP data around ref_time ±30min
                            time_diff_30 = pd.Timedelta(minutes=30)
                            LAMP_sub = LAMP_sub[(LAMP_sub['forecast_timestamp'] >= ref_time - time_diff_30) &
                                                (LAMP_sub['forecast_timestamp'] <= ref_time + time_diff_30)]

                            # If multiple rows remain, take the last one
                            if not LAMP_sub.empty:
                                LAMP_sub = LAMP_sub.sort_values('forecast_timestamp')
                                selected_row = LAMP_sub.iloc[-1]

                                weather_cols = ['temperature', 'wind_direction', 'wind_speed', 'wind_gust',
                                                'cloud_ceiling', 'visibility', 'cloud', 'lightning_prob', 'precip']

                                # Map categorical values if needed
                                for wcol in weather_cols:
                                    val = selected_row[wcol] if wcol in selected_row else np.nan
                                    if wcol == 'lightning_prob' and pd.notna(val):
                                        val = lightning_map.get(val, np.nan)
                                    if wcol == 'cloud' and pd.notna(val):
                                        val = cloud_map.get(val, np.nan)
                                    result_df[wcol] = val
                            else:
                                # No suitable LAMP data found
                                for wcol in ['temperature', 'wind_direction', 'wind_speed', 'wind_gust',
                                             'cloud_ceiling', 'visibility', 'cloud', 'lightning_prob', 'precip']:
                                    result_df[wcol] = np.nan
                        else:
                            # No LAMP data or does not meet conditions
                            for wcol in ['temperature', 'wind_direction', 'wind_speed', 'wind_gust',
                                         'cloud_ceiling', 'visibility', 'cloud', 'lightning_prob', 'precip']:
                                result_df[wcol] = np.nan

                        # Extract runway information from configs
                        cfg_features = cfg_row.to_dict()

                        def runway_count(runway_str):
                            if pd.isna(runway_str):
                                return 0
                            parts = [x.strip() for x in runway_str.split(',') if x.strip() != '']
                            return len(parts)

                        departure_runways_count = runway_count(cfg_features.get('departure_runways', None))
                        arrival_runways_count = runway_count(cfg_features.get('arrival_runways', None))

                        result_df['departure_runways'] = departure_runways_count
                        result_df['arrival_runways'] = arrival_runways_count

                        all_records.append(result_df)

                if all_records:
                    final_df = pd.concat(all_records, ignore_index=True)

                    # Fill missing values in numeric columns
                    numeric_cols = final_df.select_dtypes(include=[float,int]).columns
                    final_df[numeric_cols] = final_df[numeric_cols].interpolate(method='linear', limit_direction='both').fillna(0)

                    # Format ref_time
                    final_df['ref_time'] = pd.to_datetime(final_df['ref_time'], utc=True, errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

                    file_path = f"./data/{data_type}/{dataset_purpose}/{data_range}.csv"
                    os.makedirs(os.path.dirname(file_path), exist_ok=True)
                    final_df.to_csv(file_path, index=False)
                    print(file_path, "saved!")


./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUSER/train/2022-09-01.csv already saved, skip!
./data/FUS